In [2]:
from datetime import datetime, date
import math 
%config IPCompleter.greedy=True

## Sideral Time

Sideral time is calculated from 

In [2]:
location = {"lat": 48, "lng": 11}

In [3]:
def days_between(d1, d2):
    return abs((d2 - d1).days)

L0=99.967794687
L1=360.98564736628603
L2=2.907879e-13
L3=-5.302e-22

In [53]:
longitude=-5
hour = 23
tz = -1
deltaJulianDays = days_between(date(2000,1,1), date(2006,12,1))
deltaJulian = float(deltaJulianDays + (hour + tz) / 24)
sideralTime = ((L0 + L1 * deltaJulian + L2 * deltaJulian**2 + L3 * deltaJulian**3 - longitude) % 360) / 15
datetime(2021,12,19,math.floor(sideralTime),math.floor((sideralTime - math.floor(sideralTime))*60)).strftime("%H:%M")

'03:02'

In [99]:
longitude=11
hour = datetime.utcnow().hour - 1
tz = -1
deltaJulianDays = days_between(date(2000,1,1), date.today())
deltaJulian = float(deltaJulianDays + (hour + tz) / 24)
sideralTime = ((L0 + L1 * deltaJulian + L2 * deltaJulian**2 + L3 * deltaJulian**3 - longitude) % 360) / 15
sideralTime
deltaJulian

8023.416666666667

In [3]:
def julianDay (year, month, day):
    if month in [1, 2]:
        year -= 1;
        month += 12;
        
    A = math.floor(year / 100)
    B = 2 - A + math.floor(A / 4); 
    return math.floor(365.25 * (year + 4716)) + math.floor(30.6001 * (month + 1)) + day + B - 1524.5

In [5]:
longitude=10.9836
year = datetime.utcnow().year
month = datetime.utcnow().month
day = datetime.utcnow().day
hour = datetime.utcnow().hour
minute = datetime.utcnow().minute
second = datetime.utcnow().second
UT = hour + minute / 60 + second / 3600

deltaJulianDays = julianDay(year, month, day) - julianDay(2000, 1, 1) - 0.5
deltaJulian = deltaJulianDays + hour/24 + minute/60/24 + second/3600/24

sideralTime = ((100.46 + 0.985647 * deltaJulian + longitude + 15*UT) % 360) / 15

sideralHour = math.floor(sideralTime)
sideralMinute = math.floor((sideralTime - sideralHour) * 60)
sideralSecond = ((sideralTime - sideralHour) * 60 - sideralMinute) * 60

print(f'UTC {hour:02d}:{minute:02d}:{math.floor(second):02d} => {sideralTime} => J2000 {sideralHour:02d}:{sideralMinute:02d}:{sideralSecond:.2f}')

UTC 15:55:57 => 22.69398515785409 => J2000 22:41:38.35


In [6]:
longitude=11

deltaJulianDays = julianDay(year, month, day) - julianDay(2000, 1, 1) - 0.5
deltaJulian = deltaJulianDays + hour/24 + minute/60/24 + second/3600/24

t = deltaJulian / 36525.
sideralTime = ((280.46061837 + 360.98564736629 * deltaJulian + 0.000388 * t**2 + longitude) % 360) / 15

sideralHour = math.floor(sideralTime)
sideralMinute = math.floor((sideralTime - sideralHour) * 60)
sideralSecond = ((sideralTime - sideralHour) * 60 - sideralMinute) * 60

print(f'UTC {hour:02d}:{minute:02d}:{math.floor(second):02d} => {sideralTime} => J2000 {sideralHour:02d}:{sideralMinute:02d}:{sideralSecond:.2f}')

UTC 15:55:57 => 22.695316933716338 => J2000 22:41:43.14


In [52]:
sin = math.sin
cos = math.cos

def localSideralTime(year, month, day, hour, minute, second, longitude):
    deltaJulian = julianDay(year, month, day) - julianDay(2000, 1, 1) - 0.5 + hour/24 + minute/60/24 + second/3600/24
    julianCenturies = deltaJulian / 36525.
    # returns hours
    return ((280.46061837 + 360.98564736629 * deltaJulian + 0.000388 * julianCenturies**2 + longitude) % 360) / 15

def printSideralTime(sideralTime):
    sideralHour = math.floor(sideralTime)
    sideralMinute = math.floor((sideralTime - sideralHour) * 60)
    sideralSecond = math.floor(((sideralTime - sideralHour) * 60 - sideralMinute) * 60)
    print(f'Sideral Time (J2000) {sideralHour:02d}:{sideralMinute:02d}:{sideralSecond:02d}')
    
def rad(deg):
    return deg * math.pi / 180.

def deg(rad):
    return rad * 180 / math.pi

def horizontalToEqatorial(azimuth, altitude, latitude, localSideralTime):
    az = rad(azimuth)
    alt = rad(altitude)
    lat = rad(latitude)
    
    dec = math.asin( sin(lat) * sin(alt) + cos(lat) * cos(alt) * cos(az) )
    ra = localSideralTime - math.acos( (sin(alt) - sin(lat) * sin(dec)) / (cos(lat) * cos(dec)) ) * 180 / math.pi
    # returns all values in degrees
    return (ra, dec * 180 / math.pi)

"""
http://www.stargazing.net/kepler/altaz.html
"""
def equatorialToHorizontal(ra, declination, latitude, localSideralTime):
    hourAngle = localSideralTime - ra
    hourAngle = rad(hourAngle) if hourAngle >= 0  else hourAngle + 360
    dec = rad(declination)
    lat = rad(latitude)
    
    altitude = math.asin( sin(dec)*sin(lat) + cos(dec)*cos(lat)*cos(hourAngle) )

    A = math.acos( (sin(dec) - sin(altitude)*sin(lat))/(cos(altitude)*cos(lat)) ) 
    azimuth = A if sin(hourAngle) < 0  else 2*math.pi - A
    
    # returns all values in degrees
    return(deg(azimuth), deg(altitude))

# Sample Calculation from Alt/Az to Ra/Dec

In [39]:
# Aldebaran
Alt = 58.2746 # °
Az = 177.0994 # °
year = 2021
month = 12
day = 20
hour = 21
minute = 48
second = 6
longitude = 11 # °, East
latitude = 48

Expected Values: Ra = 4h35m56.70s = 68.98623° Dec = +16°30'31.3" = 16.5087°

In [40]:
sideralTimeHours = localSideralTime(year, month, day, hour, minute, second, longitude)
printSideralTime(sideralTimeHours)
sideralTimeDegrees = sideralTimeHours * 360 / 24
sideralTimeDegrees

Sideral Time (J2000) 04:30:53


67.72264533070847

In [20]:
Alt = 60.34
Az = 180
ST = 297.93
declination = 8.93
latitude = 38.59

ST - math.acos((sin(rad(Alt)) - sin(rad(latitude)) * sin(rad(declination)) ) / ( cos(rad(latitude)) * cos(rad(declination)) )) * 180 / math.pi

297.93

In [34]:
# Testcase #1: expected result (RA=297.92, DEC=8.93) from http://jonvoisey.net/blog/2018/07/data-converting-alt-az-to-ra-dec-example/
horizontalToEqatorial(azimuth=180, altitude=60.34, latitude=38.59, localSideralTime=297.93)

(297.93, 8.930000000000005)

In [35]:
# Testcase #2: expected result (RA=250.425, DEC=36.4667) from http://www.stargazing.net/kepler/altaz.html
horizontalToEqatorial(azimuth=269.14634, altitude=49.169122, latitude=52.5, localSideralTime=304.80762)

(250.42499131476123, 36.46666948532732)

In [42]:
# Testcase #3: expected result: (AZ=269.14634, ALT=49.169122)
equatorialToHorizontal(ra=250.425, declination=36.4667,  latitude=52.5, localSideralTime=304.80762)

(269.1463683430765, 49.16914722923827)

In [87]:
# Test case: expected result: 304.80762°
res = localSideralTime(1998, 8, 10, 23, 10, 0, -1.9166667) * 360 / 24
assert(math.isclose(res, 304.80762, rel_tol=0.0001))
res

304.80804211986833

In [88]:
# Test case: expected result: 174.77457°
res = localSideralTime(1994, 6, 16, 18, 0, 0, 0) * 360 / 24
assert(math.isclose(res, 174.77457, rel_tol=0.0001))
res

174.7711146667134

In [41]:
equatorialToHorizontal(ra=68.98623, declination=16.5087,  latitude=48, localSideralTime=67.7226)

(233.33110986873436, 47.94805962010394)